In [2]:
import sqlite3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
conn = sqlite3.connect("backup/data.db")
q = "SELECT * FROM posts"
df = pd.read_sql_query(q, conn)
df.head()

,index,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow
0,0,0.0,2018-02-14T09:50:00+0000,28.0,323288791493138_329592980862719,74.0,1.0,#1130 הדבר הכי מביך שקרה לי עד כה- ישבתי בספרי...,None,0.0,HUJI,None,None,None,103.0,POST,0.0
1,1,0.0,2018-02-14T09:40:00+0000,3.0,323288791493138_329590724196278,16.0,0.0,"#1129 גברים וגברות היום ולנטיינז, זה הרגע שלכם...",None,0.0,HUJI,None,None,None,19.0,POST,0.0
2,2,0.0,2018-02-14T09:30:00+0000,0.0,323288791493138_329592804196070,17.0,0.0,#1128 לכל מי שמתרץ את עניין הרמאויות וההעתקות ...,None,0.0,HUJI,None,None,None,17.0,POST,0.0
3,3,0.0,2018-02-14T09:20:03+0000,17.0,323288791493138_329598080862209,58.0,2.0,יוזמה נהדרת!,None,0.0,HUJI,None,None,None,77.0,POST,0.0
4,4,0.0,2018-02-14T09:20:00+0000,2.0,323288791493138_329590930862924,5.0,0.0,#1127 אולי מישהו ירים כבר את הכפפה ויפתח קבוצת...,None,0.0,HUJI,None,None,None,7.0,POST,0.0


In [4]:
df.groupby(["type"]).agg(["count"])["index"]

,count
type,
COMMENT,48733
POST,6210
TAG,39985


In [5]:
df.groupby(["source"]).agg(["count"])["index"]

,count
source,
ARIEL,6505
BGU,19149
HADASSAH,2509
HAIFA,1736
HUJI,26143
IDC,5322
TAU,17733
TECHNION,15831


In [6]:
ID_REGEX = "(#\d+|\d+#){1}"
df["id"] = df["message"].str.findall(ID_REGEX).str.get(0)
df["message"].str
df["reply_to"] = df["message"].str.findall(ID_REGEX).str.get(1)
df.head()

,index,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow,reply_to
0,0,0.0,2018-02-14T09:50:00+0000,28.0,#1130,74.0,1.0,#1130 הדבר הכי מביך שקרה לי עד כה- ישבתי בספרי...,None,0.0,HUJI,None,None,None,103.0,POST,0.0,NaN
1,1,0.0,2018-02-14T09:40:00+0000,3.0,#1129,16.0,0.0,"#1129 גברים וגברות היום ולנטיינז, זה הרגע שלכם...",None,0.0,HUJI,None,None,None,19.0,POST,0.0,NaN
2,2,0.0,2018-02-14T09:30:00+0000,0.0,#1128,17.0,0.0,#1128 לכל מי שמתרץ את עניין הרמאויות וההעתקות ...,None,0.0,HUJI,None,None,None,17.0,POST,0.0,NaN
3,3,0.0,2018-02-14T09:20:03+0000,17.0,NaN,58.0,2.0,יוזמה נהדרת!,None,0.0,HUJI,None,None,None,77.0,POST,0.0,NaN
4,4,0.0,2018-02-14T09:20:00+0000,2.0,#1127,5.0,0.0,#1127 אולי מישהו ירים כבר את הכפפה ויפתח קבוצת...,None,0.0,HUJI,None,None,None,7.0,POST,0.0,NaN


#### Get the replies for each comment

#### Get the name of the commenter (for each comment or reply)

#### Get the commenter gender

#### Get the posts a user reacted to or commented to

#### Get a list of the users commented or reacted

#### Calculate the feeling in each institute